[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

In [2]:
# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [3]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [4]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [5]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("agedistribution.json", "rt") as INFILE:
    jsondata=json.load(INFILE)


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [6]:
# Function to extract minimum age from age bands
def min_age(agerange):
    agerange = agerange.replace('+','')  # remove the + from 90+
    start = agerange.split('_to_')[0]
    return int(start)


def wrangle_data(response):
    
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    data = response['data']
    males = data[0]['males']
    females = data[0]['females']
    ageranges = [x['age'] for x in males]  # assuming males and females have the same age bands



    # Sorting age bands
    ageranges.sort(key=min_age)

    # Creating a DataFrame with age bands as index
    age_df = pd.DataFrame(index=ageranges, columns=["males", "females", "total"])

    # Populating the DataFrame with the male and female data
    for entry in males:
        ageband = entry['age']
        age_df.loc[ageband, 'males'] = entry['value']

    for entry in females:
        ageband = entry['age']
        age_df.loc[ageband, 'females'] = entry['value']

    age_df['total'] = age_df['males'] + age_df['females']

    return age_df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [7]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=nation',
    'areaName=England'
    ]


# values here are the names of the PHE metrics
    structure = {
        "males": "maleCases",
        "females": "femaleCases"
        }
    api = Cov19API(filters=filters, structure=structure)
    
    return api.get_json() # return data read from the API

In [8]:
access= access_api()
#print(access)

{'data': [{'males': [{'age': '15_to_19', 'rate': 40074.4, 'value': 641772}, {'age': '0_to_4', 'rate': 14279.3, 'value': 237364}, {'age': '80_to_84', 'rate': 23092.6, 'value': 147797}, {'age': '60_to_64', 'rate': 28298.4, 'value': 443858}, {'age': '65_to_69', 'rate': 23719.5, 'value': 319671}, {'age': '85_to_89', 'rate': 29627.8, 'value': 105529}, {'age': '45_to_49', 'rate': 36240.9, 'value': 653498}, {'age': '20_to_24', 'rate': 38487.8, 'value': 689587}, {'age': '30_to_34', 'rate': 41130.1, 'value': 788223}, {'age': '90+', 'rate': 40660.5, 'value': 68939}, {'age': '50_to_54', 'rate': 33465.7, 'value': 639635}, {'age': '25_to_29', 'rate': 39872.3, 'value': 767308}, {'age': '70_to_74', 'rate': 20868.8, 'value': 280461}, {'age': '5_to_9', 'rate': 32159.3, 'value': 583486}, {'age': '40_to_44', 'rate': 41128.4, 'value': 711631}, {'age': '35_to_39', 'rate': 40012.3, 'value': 741415}, {'age': '75_to_79', 'rate': 23424.9, 'value': 218806}, {'age': '10_to_14', 'rate': 46546.9, 'value': 820097},

In [9]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='PANIC', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='danger', description='PANIC', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [10]:


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back
    
    

In [11]:
import ipywidgets as widgets

# SelectMultiple for data columns
data_columns = widgets.SelectMultiple(
    options=['males', 'females', 'total'],  # replace with your column names
    value=['males'],
    description='Data:',
    disabled=False
)

# RadioButton for selecting scale type
scale_type = widgets.RadioButtons(
    options=['linear', 'log'],
    value='linear',
    description='Scale:'
)
def update_graph(columns, scale):
    plt.figure(figsize=(10, 6))
    for column in columns:
        plt.plot(df[column], label=column)
        plt.yscale(scale)
        plt.legend()
        plt.show()
    # Use 'columns' to filter/select your data
    # Use 'scale' to set the graph's y-axis scale type
    # Plot your graph here

from ipywidgets import interactive

interactive_plot = interactive(update_graph, columns=data_columns, scale=scale_type)
display(interactive_plot)

interactive(children=(SelectMultiple(description='Data:', index=(0,), options=('males', 'females', 'total'), v…

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).